In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
from splinter import Browser
from tqdm import tqdm_notebook 

In [2]:
#setting up path to chrome driver for splinter
executable_path={'executable_path':'chromedriver.exe'}

In [3]:
#stats scrape - get list of urls to scrape to get team stats
url_stats='http://www.milb.com/milb/stats/'
response_2=requests.get(url_stats)
soup_2=bs(response_2.content, 'lxml')

team_urls=[]
team_aaa_international=soup_2.find('li', id='league_117_bg')
teams_list_international=team_aaa_international.find('div', class_='teams-list')

international_team_urls=teams_list_international.find_all('a', href=True)
team_urls={}
for i, link in enumerate(international_team_urls):
    team_name=link.text
    team_urls[team_name]=[link['href'], i]

In [4]:
batting_columns=['Name',
                 'id',
                 'POS',
                 'G',
                 'AB',
                 'R',
                 'H',
                 '2B',
                 '3B',
                 'HR',
                 'RBI',
                 'TB',
                 'BB',
                 'SO',
                 'SB',
                 'CS',
                 'OBP',
                 'SLG',
                 'AVG',
                 'OPS']

pitching_columns=['Name',
                  'id',
                  'W',
                  'L',
                  'ERA',
                  'G',
                  'GS',
                  'CG',
                  'SHO',
                  'SV',
                  'IP',
                  'H',
                  'R',
                  'ER',
                  'HR',
                  'HB',
                  'BB',
                  'SO',
                  'WHIP',
                  'HLD',
                  'GF']

In [5]:
pitching_dfs=[]
batting_dfs=[]

for team_name, values in tqdm_notebook(team_urls.items()):
    
    url=values[0]
    team_index=values[1]
    
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)

    html = browser.html
    soup = bs(html, 'lxml')
    
    ##### BATTING TABLE #########
    batting_table=soup.find('div', id='stats_data_ibp_hitting')
    batting_trs=batting_table.find_all('tr')

    batting_stats=[]
    for tr in batting_trs:
        td=tr.find_all('td')
        row=[tr.text for tr in td]
        batting_stats.append(row)
   
    exec('df_batting_{}=pd.DataFrame(batting_stats, columns=batting_columns)'.format(team_index))
    exec('df_batting_{}.dropna(inplace=True)'.format(team_index))
    exec('df_batting_{}["Team"]=team_name'.format(team_index))
    exec('batting_dfs.append(df_batting_{})'.format(team_index))

    ####### PITCHING TABLE #################
    pitching_table=soup.find('div', id='stats_data_ibp_pitching')
    pitching_trs=pitching_table.find_all('tr')

    pitching_stats=[]
    for tr in pitching_trs:
        td=tr.find_all('td')
        row=[tr.text for tr in td]
        pitching_stats.append(row)

    exec('df_pitching_{}=pd.DataFrame(pitching_stats, columns=pitching_columns)'.format(team_index))
    exec('df_pitching_{}.dropna(inplace=True)'.format(team_index))
    exec('df_pitching_{}["Team"]=team_name'.format(team_index))
    exec('pitching_dfs.append(df_pitching_{})'.format(team_index))
    
    #close browser
    browser.quit()


In [6]:
df_pitching=pd.concat(pitching_dfs)
df_batting=pd.concat(batting_dfs)

df_batting.reset_index(inplace=True, drop=True)
df_pitching.reset_index(inplace=True, drop=True)

df_batting.to_csv('./Data_Output/batting.csv', index=False)
df_pitching.to_csv('./Data_Output/pitching.csv', index=False)